# Function Calling in OpenAI

In [26]:
import os
import openai
from dotenv import load_dotenv

In [27]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [28]:
prompt = "Hello, How are you?"

response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

Hello! I'm here and ready to help. How can I assist you today?


https://rapidapi.com/hub

In [29]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      'x-rapidapi-key': "156877d1e1msh6806b57dfec44b9p1bd2c8jsn796f2ff15790",
      'x-rapidapi-host':  os.getenv("RAPIDAPI_KEY")
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()


In [30]:
response = get_current_weather("Delhi")

{'messages': "The host you've provided is invalid. If you have difficulties, contact the RapidAPI support team, support@rapidapi.com"}


In [31]:
response

{'messages': "The host you've provided is invalid. If you have difficulties, contact the RapidAPI support team, support@rapidapi.com"}

In [32]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [33]:

user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})

completion=openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages= messages
    
)

In [34]:
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [35]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [36]:
user_message="What is the temperature of Bangalore"

messages.append({"role": "user", "content": user_message})

completion=openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
    
)

In [37]:
# messages

In [38]:
# completion

In [39]:
print(completion.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None)


In [40]:
response=completion.choices[0].message

In [41]:
response

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None)

In [42]:
response.function_call.name

'get_current_weather'

In [43]:
function_name=response.function_call.name
print(function_name)


get_current_weather


In [44]:
import json
location=eval(response.function_call.arguments)
print(location)

{'location': 'Bangalore'}


In [45]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Bangalore'}]

In [46]:
# When adding the function response:
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": json.dumps(location)  # Make sure this is a string, not an object
})

# Now try the API call again
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [48]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Bangalore'},
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": "Bangalore"}'}]

In [47]:
print(second_response.choices[0].message)

ChatCompletionMessage(content='The current temperature in Bangalore is 27°C.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
